In [6]:
import os
import sys
import glob
import shutil
import random
import pickle
import numpy as np
from PIL import Image
import time
import copy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torch.utils.data
from torchvision.models import inception, inception_v3
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# import my functions:
from tile_train_functions import*

# Set it to use GPU1
torch.cuda.set_device(1)
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/Data/camelyon/src
True
1


In [2]:
model_list = [resnet34, resnet34, resnet34, resnet34, resnet34, resnet34, resnet34, resnet34, resnet34,resnet34, resnet34, resnet34]
model_name = ['resnet34_scratch_no_samp_0', 'resnet34_scratch_no_samp_1', 'resnet34_scratch_no_samp_2',
    'resnet34_scratch_no_samp_3', 'resnet34_scratch_no_samp_4', 'resnet34_scratch_no_samp_5',
    'resnet34_scratch_no_samp_6', 'resnet34_scratch_no_samp_7', 'resnet34_scratch_no_samp_8',
    'resnet34_scratch_no_samp_9', 'resnet34_scratch_no_samp_10', 'resnet34_scratch_no_samp_11']  

save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/models'

PATH = '/media/rene/Data/camelyon_out/tiles_224_100t/'
num_workers = 4
batch_size=64
sz=224
dataloaders_valid, dataset_sizes_valid = make_batch_gen(PATH, batch_size, num_workers, valid_name='valid')
dataloaders_test, dataset_sizes_test = make_batch_gen(PATH, batch_size, num_workers, valid_name='test')

criterion = nn.CrossEntropyLoss()

results = {}

for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    print('Validation: ', model_name[idx])
    valid_loss, valid_acc = eval_model(model_ft, dataloaders_valid['valid'], dataset_sizes_valid['valid'], criterion)
    
    print('Test: ', model_name[idx])
    test_loss, test_acc = eval_model(model_ft, dataloaders_test['test'], dataset_sizes_test['test'], criterion)
    results[model_name[idx]] = [valid_acc, test_acc]
    
f = open(os.path.join(save_path, "result_resnet34_scratch_no_samp.pkl"),"wb")
pickle.dump(results,f)
f.close()

KeyboardInterrupt: 

In [ ]:
results2 = {key:value[0] for (key,value) in results.items()}

import operator
sorted_x = sorted(results2.items(), key=operator.itemgetter(1))

print(sorted_x)

## Best valid acc

In [ ]:
sz = 224

model_list = [resnet34, resnet34, resnet34]
model_name = ['resnet34_scratch_no_samp_3', 'resnet34_scratch_no_samp_7', 'resnet34_scratch_no_samp_1']    
save_path = '/media/rene/Data/camelyon_out/tiles_224_100t/models'

batch_size = 32
num_workers = 4
PATH = '/media/rene/Data/camelyon_out/tiles_224_100t'
dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers)


model_list_ft = []    
for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=False)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()
    
    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    model_list_ft.append(model_ft)
    
fusion_model = WeightedSum(num_input=6)
fusion_model = fusion_model.cuda()

num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(fusion_model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

model = train_fusion_model(fusion_model, model_list_ft, 
                    criterion, optimizer_ft, exp_lr_scheduler, num_epochs,
                     dataloaders, dataset_sizes)

torch.save(model.state_dict(), os.path.join(save_path, 'weighted_sum_fusion_3resnet34_nosamp_b3'))

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0/9
----------


/home/rene/miniconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


train Loss: 0.2959 Acc: 0.8844


 10%|█         | 1/10 [16:02<2:24:25, 962.84s/it]

valid Loss: 0.1696 Acc: 0.9296
saving model with acc  0.9295808383233533
Epoch 1/9
----------
train Loss: 0.1606 Acc: 0.9359


 20%|██        | 2/10 [32:06<2:08:26, 963.25s/it]

valid Loss: 0.1720 Acc: 0.9286
Epoch 2/9
----------
train Loss: 0.1622 Acc: 0.9356


 30%|███       | 3/10 [48:09<1:52:22, 963.26s/it]

valid Loss: 0.1722 Acc: 0.9274
Epoch 3/9
----------


In [18]:
save_path = '/media/rene/Data/data/camelyon_out/tiles_224_100t/models'
PATH = '/media/rene/Data/data/camelyon_out/tiles_224_100t/'
sz=224

model_list = [resnet34, resnet34, resnet34]
model_name = ['resnet34_scratch_no_samp_3', 'resnet34_scratch_no_samp_7', 'resnet34_scratch_no_samp_2']    

dataloaders, dataset_sizes = make_batch_gen(PATH, batch_size, num_workers, 
                                                        valid_name='valid', test_name = 'test')
criterion = nn.CrossEntropyLoss()

model_list_ft = []    
for idx, model_arch in enumerate(model_list):
    # get the proper model architecture
    model_ft = model_arch(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)
    model_ft = model_ft.cuda()

    # load the saved weights
    model_ft.load_state_dict(torch.load(os.path.join(save_path, model_name[idx])))
    model_list_ft.append(model_ft)

batch_size = 8
num_workers = 4

fusion_model = WeightedSum(num_input=6)
fusion_model.load_state_dict(torch.load(os.path.join(save_path, 'weighted_sum_fusion_3resnet34_nosamp')))
fusion_model = fusion_model.cuda() 

# print('Validation weighted_sum_fusion_3resnet34_scratch_nosamp: ')
# eval_fusion_model(fusion_model, model_list_ft, dataloaders_valid['valid'], dataset_sizes_valid['valid'], criterion)

# print('Test weighted_sum_fusion_3resnet34_scratch_nosamp: ')
# eval_fusion_model(fusion_model, model_list_ft, dataloaders_test['test'], dataset_sizes_test['test'], criterion)

In [21]:
all_labels, all_preds = get_preds_fusion(fusion_model, model_list_ft, dataloaders['valid'], dataset_sizes['valid'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Validation sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

all_labels, all_preds = get_preds_fusion(fusion_model, model_list_ft, dataloaders['test'], dataset_sizes['test'])
metrics = get_metrics_bin(all_labels, all_preds)
print(f'Test sensitivity: {metrics["TPR"]}, specificity: {metrics["TNR"]}, Acc: {metrics["ACC"]}')

Validation sensitivity: 0.9144768275203058, specificity: 0.8974543707973103, Acc: 0.9059880239520958
Test sensitivity: 0.8215632324467543, specificity: 0.9118995077236463, Acc: 0.8708497337346608
